In [36]:
import os
from shutil import copyfile
import glob
import zipfile
import math
import random
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers


#from keras import optimizers
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from glob import glob

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download -d ashishsaxena2209/animal-image-datasetdog-cat-and-panda

100% 376M/376M [00:03<00:00, 123MB/s]
100% 376M/376M [00:03<00:00, 116MB/s]


In [11]:
zip_ref = zipfile.ZipFile('/content/animal-image-datasetdog-cat-and-panda.zip')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
os.mkdir('/tmp/dataset')
os.mkdir('/tmp/dataset/train')
os.mkdir('/tmp/dataset/test')
os.mkdir('/tmp/dataset/valid')

os.mkdir('/tmp/dataset/train/cats')
os.mkdir('/tmp/dataset/train/dogs')
os.mkdir('/tmp/dataset/train/panda')
os.mkdir('/tmp/dataset/valid/cats')
os.mkdir('/tmp/dataset/valid/dogs')
os.mkdir('/tmp/dataset/valid/panda')
  

In [19]:
def split_data(SOURCE, TRAINING, VALIDATION ,TESTING,SPLIT_SIZE):
  total_imgs = len(os.listdir(SOURCE))
  training_imgs = math.ceil(SPLIT_SIZE*total_imgs)
  validation_img = total_imgs - training_imgs
  testing_imgs  = total_imgs-training_imgs
  print(total_imgs,training_imgs,testing_imgs)
  for image in random.sample(os.listdir(SOURCE),training_imgs):
    copyfile(SOURCE+image,TRAINING+image)
  for image in random.sample(os.listdir(SOURCE),validation_img):
    copyfile(SOURCE+image,VALIDATION+image)
  for image in random.sample(os.listdir(SOURCE),testing_imgs):
    copyfile(SOURCE+image,TESTING+image)

split_size = 0.8
cat_source_dir = '/tmp/animals/cats/'
dog_source_dir = '/tmp/animals/dogs/'
panda_source_dir = '/tmp/animals/panda/'

training_cat = '/tmp/dataset/train/cats/'
training_dog = '/tmp/dataset/train/dogs/'
training_panda = '/tmp/dataset/train/panda/'

validation_cat = '/tmp/dataset/valid/cats/'
validation_dog = '/tmp/dataset/valid/dogs/'
validation_panda = '/tmp/dataset/valid/panda'

testing = '/tmp/dataset/test/'

split_data(cat_source_dir ,training_cat,validation_cat,testing , split_size)
split_data(dog_source_dir ,training_dog,validation_dog,testing , split_size)
split_data(panda_source_dir ,training_panda,validation_panda,testing , split_size)

1000 800 200
1000 800 200
1000 800 200


In [32]:
from keras.preprocessing.image import ImageDataGenerator
base_renet =tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(32,32,3),
    classes=3,
)

In [33]:
for layer in base_renet.layers:
  layer.trainable = False

In [42]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
flat1 = Flatten()(base_renet.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(3, activation='softmax')(class1)
# define new model
model = Model(inputs=base_renet.inputs, outputs=output)

In [43]:
base_renet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [44]:
optimizer=Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [45]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
#Data Augmentation Function: Let's define an instance of the ImageDataGenerator class and set the parameters
# We have to instantiate for the Train,Validation and Test datasets
train_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip= True,
                                    zoom_range=.1) 

In [49]:
class_subset = sorted(os.listdir('/tmp/dataset/train/'))
BATCH_SIZE = 64
train_path = '/tmp/dataset/train/'
valid_path = '/tmp/dataset/valid'
test_path = '/tmp/dataset/test/'

traingen = train_generator.flow_from_directory(train_path,
                                               target_size=(32, 32),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=5)

validgen = train_generator.flow_from_directory(valid_path,
                                               target_size=(32, 32),
                                               class_mode = 'categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=3)

testgen = test_generator.flow_from_directory(test_path,
                                             target_size=(32, 32),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=4)

Found 2400 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


In [51]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 10

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best_50epochs.hdf5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=2)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/10
37/37 [==============================] - 16s 419ms/step - loss: 0.7165 - accuracy: 0.6862
Epoch 2/10
37/37 [==============================] - 15s 407ms/step - loss: 0.6017 - accuracy: 0.7324
Epoch 3/10
37/37 [==============================] - 15s 405ms/step - loss: 0.5275 - accuracy: 0.7568
Epoch 4/10
37/37 [==============================] - 15s 402ms/step - loss: 0.4805 - accuracy: 0.8005
Epoch 5/10
37/37 [==============================] - 15s 407ms/step - loss: 0.4470 - accuracy: 0.8069
Epoch 6/10
37/37 [==============================] - 15s 408ms/step - loss: 0.4077 - accuracy: 0.8249
Epoch 7/10
37/37 [==============================] - 15s 408ms/step - loss: 0.4050 - accuracy: 0.8296
Epoch 8/10
37/37 [==============================] - 15s 405ms/step - loss: 0.3675 - accuracy: 0.8450
Epoch 9/10
37/37 [==============================] - 15s 406ms/step - loss: 0.3729 - accuracy: 0.8438
Epoch 10/10
37/37 [==============================] - 15s 407ms/step - loss: 0.3343 - accura